In [1]:
import os
import pathlib
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import nvitop
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

# os.environ["TF_GPU_THREAD_MODE"] = "gpu_private"
tf.distribute.OneDeviceStrategy(device="/gpu:0")
# policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
# tf.keras.mixed_precision.experimental.set_policy(policy)

In [2]:
datadir = pathlib.Path(
    r"C:\Users\jadav\Desktop\Tensorflow notebooks\cats-v-dogs\training"
)
valdir = pathlib.Path(
    r"C:\Users\jadav\Desktop\Tensorflow notebooks\cats-v-dogs\testing"
)
# import os

# datadir = os.getcwd()
# datadir = datadir + '/cats-v-dogs/testing/cats'
# for fn in os.listdir(datadir):
#     if fn[-3:-1] != 'jp':
#         print(fn)
AUTOTUNE = tf.data.AUTOTUNE

logdir = pathlib.Path.cwd() / "logs"

if logdir.exists():
    !rmdir /q/s logs

In [3]:
# import os
# import cv2
# from PIL import Image
# def check_images( s_dir, ext_list):
#     bad_images=[]
#     bad_ext=[]
#     s_list= os.listdir(s_dir)
#     for klass in s_list:
#         klass_path=os.path.join (s_dir, klass)
#         print ('processing class directory ', klass)
#         if os.path.isdir(klass_path):
#             file_list=os.listdir(klass_path)
#             for f in file_list:
#                 f_path=os.path.join (klass_path,f)
#                 index=f.rfind('.')
#                 ext=f[index+1:].lower()
#                 if ext not in ext_list:
#                     print('file ', f_path, ' has an invalid extension ', ext)
#                     bad_ext.append(f_path)
#                     os.remove(f_path)
#                 if os.path.isfile(f_path):
#                     try:
#                         img=cv2.imread(f_path)
#                         shape=img.shape
#                     except:
#                         print('file ', f_path, ' is not a valid image file')
#                         bad_images.append(f_path)
# #                         img = Image.open(f_path)
# #                         img = img.save(f_path + ".jpeg")
#                         os.remove(f_path)
#                 else:
#                     print('*** fatal error, you a sub directory ', f, ' in class directory ', klass)
#         else:
#             print ('*** WARNING*** you have files in ', s_dir, ' it should only contain sub directories')
#     return bad_images, bad_ext
# ext_list = ["jpg"]
# check_images(valdir, ext_list)

In [4]:
image_count = len(list(datadir.glob("*/*.jpg")))
print(image_count)
batch_size = 128
img_height = 150
img_width = 150

22289


In [5]:
list_ds = tf.data.Dataset.list_files(str(datadir / "*/*.jpg"), shuffle=False)
val_ds = tf.data.Dataset.list_files(str(valdir / "*/*.jpg"), shuffle=False)
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)
# train_files = tf.data.Dataset.from_tensor_slices(list_ds)
# val_files = tf.data.Dataset.from_tensor_slices(val_ds)
# print(train_files, val_files)

NameError: name 'train_files' is not defined

In [ ]:
for f in list_ds.take(5):
    print(f.numpy())
print("validation")
for f in val_ds.take(5):
    print(f.numpy())

In [ ]:
class_names = np.array(sorted([item.name for item in datadir.glob("*")]))
print(class_names)

In [ ]:
# val_size = int(image_count * .2)
# train_ds = list_ds.skip(val_size)
# val_ds = list_ds.take(val_size)
train_ds = list_ds

In [ ]:
print(tf.data.experimental.cardinality(train_ds).numpy())
print(tf.data.experimental.cardinality(val_ds).numpy())

In [ ]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    one_hot = parts[-2] == class_names
    return tf.argmax(one_hot)

In [ ]:
def decode_img(img):
    img = tf.image.decode_jpeg(img, channels=3)
    return img  # tf.image.resize(img, [img_height, img_width])

In [ ]:
def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [ ]:
train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
for image, label in train_ds.take(1):
    print("Image Shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

In [ ]:
data_augmentation = tf.keras.Sequential(
    [
        #         layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.2),
        layers.experimental.preprocessing.RandomWidth(0.01, interpolation="bilinear"),
        layers.experimental.preprocessing.RandomHeight(0.01, interpolation="bilinear"),
    ]
)

resize_and_rescale = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.Resizing(img_height, img_width),
        layers.experimental.preprocessing.Rescaling(1.0 / 255),
    ]
)


def prepare(ds, shuffle=False, augment=False):
    # Resize and rescale all datasets
    ds = ds.map(lambda x, y: (resize_and_rescale(x), y), num_parallel_calls=AUTOTUNE)
    ds.cache()

    if shuffle:
        ds = ds.shuffle(1000)

    # Batch all datasets
    ds = ds.batch(batch_size)

    # Use data augmentation only on the training set
    if augment:
        ds = ds.map(
            lambda x, y: (data_augmentation(x, training=True), y),
            num_parallel_calls=AUTOTUNE,
        )
    #     if interleave:
    #         ds = ds.interleave(list_ds.map(process_path, num_parallel_calls=AUTOTUNE), cycle_length=4, num_parallel_calls=AUTOTUNE)
    # Use buffered prefecting on all datasets
    return ds.prefetch(buffer_size=AUTOTUNE)


train_ds = prepare(train_ds, shuffle=True, augment=True)
val_ds = prepare(val_ds)

In [ ]:
# def configure_for_performance(ds):
#     ds = ds.cache()
#     ds = ds.shuffle(buffer_size=1000)
#     ds = ds.batch(batch_size)
#     ds = ds.prefetch(buffer_size=AUTOTUNE)
#     return ds
# train_ds = configure_for_performance(train_ds)
# val_ds = configure_for_performance(val_ds)

In [ ]:
img_batch, label_batch = next(iter(train_ds))

plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(img_batch.numpy()[i])
    label = label_batch[i]
    plt.title(class_names[label])
    plt.axis("off")

In [ ]:
# model = tf.keras.models.Sequential(
#     [
#         tf.keras.layers.Conv2D(
#             32, (3, 3), input_shape=(150, 150, 3), activation="relu"
#         ),
#         tf.keras.layers.MaxPooling2D(2, 2),
#         tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
#         tf.keras.layers.MaxPooling2D(2, 2),
# #         tf.keras.layers.Dropout(0.4),
#         tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
#         tf.keras.layers.MaxPooling2D(2, 2),
#         tf.keras.layers.Conv2D(256, (3, 3), activation="relu"),
#         tf.keras.layers.MaxPooling2D(2, 2),
#         tf.keras.layers.Dropout(0.2),
#         tf.keras.layers.Conv2D(512, (3, 3), activation="relu"),
#         #         tf.keras.layers.Maxpooling2D(2,2),
#         tf.keras.layers.Flatten(),
#         tf.keras.layers.Dropout(0.2),
#         tf.keras.layers.Dense(120, activation="relu"),
#         tf.keras.layers.Dense(1, activation="relu"),
#     ]
# )

# model.compile(
#     optimizer=RMSprop(lr=0.001), loss="binary_crossentropy", metrics=["accuracy"]
# )

model = tf.keras.models.Sequential(
    [
        # Note the input shape is the desired size of the image 150x150 with 3 bytes color
        tf.keras.layers.Conv2D(
            16, (3, 3), activation="relu", input_shape=(150, 150, 3)
        ),
        tf.keras.layers.Conv2D(16, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
        tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
        #         tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(512, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        # Flatten the results to feed into a DNN
        tf.keras.layers.Flatten(),
        # 512 neuron hidden layer
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

model.compile(
    optimizer=RMSprop(lr=0.001), loss="binary_crossentropy", metrics=["accuracy"]
)

model.summary()

In [ ]:
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=logs, histogram_freq=1, profile_batch="500,520"
)

callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", mode="auto", patience=4, restore_best_weights=True
)

In [ ]:
history = model.fit(
    train_ds,
    epochs=50,
    verbose=1,
    validation_data=val_ds,
    callbacks=[callback, tboard_callback],
)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=logs

In [ ]:
import numpy as np

# from google.colab import files
from keras.preprocessing import image

# uploaded=files.upload()

# for fn in uploaded.keys():

# predicting images
#   path='/content/' + fn
path = "C:/Users/jadav/Desktop/Tensorflow notebooks/cat-dog-val/"
for fn in os.listdir(path):
    img = image.load_img(path + fn, target_size=(150, 150))

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])

    classes = model.predict(images, batch_size=32)

    if classes[0] > 0:
        print(fn + " is a dog")

    else:
        print(fn + " is a cat")
# print(len(right_answers),len(wrong_answers), right_answers, wrong_answers)